<a href="https://colab.research.google.com/github/AnshumanAI/Projects/blob/Openhack/Copy_of_Saaransh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'topic-labeled-news-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F836401%2F1428159%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240216%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240216T190136Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D229b28a0137779273f8517210eaeee9811ee73551711cc0aa042045c3ded357a858cffc6b4457a4aaa87e6efa924f93d5f3cacca8d77c00ccfbcfffa97884d7957a5f99f185712674b67da2a3ab993d54af1ce04b3964f30a40fee3f8429cf70d2a1b65d8d49b07d6302d075d5e0f55026f7c7a21ec070f72d66b8b30675d82155b9a25e49676f0f166f99262839f5da8532e13782690a264987b8cbc44dc584d80271478a45e72255c393ae95696d4c01e81a6e2f6195528d86afdfda1cc743b9469a4cad06964e4d75053581fc6f4bc186d996e15c8e2d98fa75a3461277869629ad6f909764c70a3ff64e299a459708adfc94f765e4ea6268d47f8c7e318a,news-category-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F32526%2F4243451%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240216%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240216T190136Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D04adc458e59fa24f05105e5b4a5d560a24d1f7f275f2b1e90d6c95d07ff260e4720945863b37d67d29053d090db6cfcbe1baa5ebf2b35a60c32fe79b7f9dac28f1cfb3a2f93487862cdf5ef34b58c8071031c6d4dcd17357a9004e1bf1dfbe2d4881639d33de989cafb470bb296a0d39b0914fcfbc73e600c0b7249c38a9ef664b1c4c8621493465071ea1ab1df18d4ff1db4663c1319720c27ae370b661046ce90ba1d37284e559773109fc66b0b2c34a4d2f25e5d7e4c36dd5a9456e696a626c0bfa58b3ef38b6614c84a1ebc6f703136c85b60cd32ecbad6c48dbca72c93d732fbfaafda6af52fd80f018704036b70cc20fbb2ead0d2253a4e35c766da838,global-news-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3990170%2F7105651%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240216%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240216T190136Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4a4435729e006f3386e5fd90687ac99c5648664916169b2684dd70a08872c63452845d3058cc437fb804977decc6d2f60ba56f6c477588004139a1c106684a1d6e56337a26f591887b22ae8e188af97c9fab3c0ed723acd08bc83fbc56ec181782959d5afcffab770aa1a9fa0158195d86db9dfa797f20402bd421bc53686f1bb4680aca44fd315a06f0f91be986d9a08a8bd7cd19095a54e938a93e92400be50c9a880975cd82220abd5c82dd55d1d52861cba7e1f2a34c33e9fd7855d01a0b1cf967ee8c2e53c3d2a5c8ba9a69997e972d3d22bdae3c557d5cf870d4e1e44d52dd2a71d5d2e206cfffbf4b4c2e3a61b069549f22e295c7c279faf2e8214fd8,bart/keras/bart_large_en_cnn/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F4674%2F6053%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240216%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240216T190136Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2e75dbf3e4e5233b4b29b318a732d2a268995b9a77ec47b8c481fc446fa95b4b297962972e7f1c14be41d25e9e09247ca25a389a051c20e97220881f57d39c77ec82b76ea07e1f42dc01dcdc7f196c0c4d9e1bc301339da3733998f72075ce49ef6334025498a433c8880a2363de78c2da207caf360feba8df3e93bd6272c008e6f09236e302a796d09f473521f89a222fabdeef9892455ac14ae5d23e97a8e0e6c8a885345aee8c9007ac942385e696952fb5fe1c0058b87990a68147e4ecd4664c3cbc50582954db0fb95a8d03b3a5e7f04f9b7531943372585178af752645a520fbfe75a8d0a4e01906bdf554bf47d14bf9392d650525a6b2ba7643b508e1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 9909326 bytes downloaded
Downloaded and uncompressed: topic-labeled-news-dataset
[==================================================] 27829769 bytes downloaded
Downloaded and uncompressed: news-category-dataset
[==================================================] 419253490 bytes downloaded
Downloaded and uncompressed: global-news-dataset
[==================================================] 971105055 bytes downloaded
Downloaded and uncompressed: bart/keras/bart_large_en_cnn/2
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bart/keras/bart_large_en_cnn/2/model.weights.h5
/kaggle/input/bart/keras/bart_large_en_cnn/2/config.json
/kaggle/input/bart/keras/bart_large_en_cnn/2/metadata.json
/kaggle/input/bart/keras/bart_large_en_cnn/2/tokenizer.json
/kaggle/input/bart/keras/bart_large_en_cnn/2/assets/tokenizer/merges.txt
/kaggle/input/bart/keras/bart_large_en_cnn/2/assets/tokenizer/vocabulary.json
/kaggle/input/topic-labeled-news-dataset/labelled_newscatcher_dataset.csv
/kaggle/input/news-category-dataset/News_Category_Dataset_v3.json
/kaggle/input/global-news-dataset/data.csv
/kaggle/input/global-news-dataset/rating.csv
/kaggle/input/global-news-dataset/raw-data.csv


In [ ]:
dat= pd.read_csv("/kaggle/input/global-news-dataset/data.csv")

In [ ]:
dat.head()

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,full_content
0,89541,NaN,International Business Times,Paavan MATHEMA,UN Chief Urges World To 'Stop The Madness' Of ...,UN Secretary-General Antonio Guterres urged th...,https://www.ibtimes.com/un-chief-urges-world-s...,https://d.ibtimes.com/en/full/4496078/nepals-g...,2023-10-30 10:12:35.000000,UN Secretary-General Antonio Guterres urged th...,Nepal,UN Secretary-General Antonio Guterres urged th...
1,89542,NaN,Prtimes.jp,NaN,RANDEBOOよりワンランク上の大人っぽさが漂うニットとベストが新登場。,[株式会社Ainer]\nRANDEBOO（ランデブー）では2023年7月18日(火)より公...,https://prtimes.jp/main/html/rd/p/000000147.00...,https://prtimes.jp/i/32220/147/ogp/d32220-147-...,2023-10-06 04:40:02.000000,"RANDEBOO2023718()WEB2023 Autumn Winter \n""Nepa...",Nepal,NaN
2,89543,NaN,VOA News,webdesk@voanews.com (Agence France-Presse),UN Chief Urges World to 'Stop the Madness' of ...,UN Secretary-General Antonio Guterres urged th...,https://www.voanews.com/a/un-chief-urges-world...,https://gdb.voanews.com/01000000-0a00-0242-60f...,2023-10-30 10:53:30.000000,"Kathmandu, Nepal UN Secretary-General Antonio...",Nepal,NaN
3,89545,NaN,The Indian Express,Editorial,Sikkim warning: Hydroelectricity push must be ...,Ecologists caution against the adverse effects...,https://indianexpress.com/article/opinion/edit...,https://images.indianexpress.com/2023/10/edit-...,2023-10-06 01:20:24.000000,At least 14 persons lost their lives and more ...,Nepal,At least 14 persons lost their lives and more ...
4,89547,NaN,The Times of Israel,Jacob Magid,"200 foreigners, dual nationals cut down in Ham...","France lost 35 citizens, Thailand 33, US 31, U...",https://www.timesofisrael.com/200-foreigners-d...,https://static.timesofisrael.com/www/uploads/2...,2023-10-27 01:08:34.000000,"Scores of foreign citizens were killed, taken ...",Nepal,NaN


In [ ]:
dat.drop("source_id", axis=1)

,article_id,source_name,author,title,description,url,url_to_image,published_at,content,category,full_content
0,89541,International Business Times,Paavan MATHEMA,UN Chief Urges World To 'Stop The Madness' Of ...,UN Secretary-General Antonio Guterres urged th...,https://www.ibtimes.com/un-chief-urges-world-s...,https://d.ibtimes.com/en/full/4496078/nepals-g...,2023-10-30 10:12:35.000000,UN Secretary-General Antonio Guterres urged th...,Nepal,UN Secretary-General Antonio Guterres urged th...
1,89542,Prtimes.jp,NaN,RANDEBOOよりワンランク上の大人っぽさが漂うニットとベストが新登場。,[株式会社Ainer]\nRANDEBOO（ランデブー）では2023年7月18日(火)より公...,https://prtimes.jp/main/html/rd/p/000000147.00...,https://prtimes.jp/i/32220/147/ogp/d32220-147-...,2023-10-06 04:40:02.000000,"RANDEBOO2023718()WEB2023 Autumn Winter \n""Nepa...",Nepal,NaN
2,89543,VOA News,webdesk@voanews.com (Agence France-Presse),UN Chief Urges World to 'Stop the Madness' of ...,UN Secretary-General Antonio Guterres urged th...,https://www.voanews.com/a/un-chief-urges-world...,https://gdb.voanews.com/01000000-0a00-0242-60f...,2023-10-30 10:53:30.000000,"Kathmandu, Nepal UN Secretary-General Antonio...",Nepal,NaN
3,89545,The Indian Express,Editorial,Sikkim warning: Hydroelectricity push must be ...,Ecologists caution against the adverse effects...,https://indianexpress.com/article/opinion/edit...,https://images.indianexpress.com/2023/10/edit-...,2023-10-06 01:20:24.000000,At least 14 persons lost their lives and more ...,Nepal,At least 14 persons lost their lives and more ...
4,89547,The Times of Israel,Jacob Magid,"200 foreigners, dual nationals cut down in Ham...","France lost 35 citizens, Thailand 33, US 31, U...",https://www.timesofisrael.com/200-foreigners-d...,https://static.timesofisrael.com/www/uploads/2...,2023-10-27 01:08:34.000000,"Scores of foreign citizens were killed, taken ...",Nepal,NaN
...,...,...,...,...,...,...,...,...,...,...,...
105370,781108,The Indian Express,PTI,"Have done no wrong, only did party work, says ...",The High Court today allowed Shivakumar to wit...,https://indianexpress.com/article/cities/banga...,https://images.indianexpress.com/2023/11/DK-Sh...,2023-11-29 10:57:22,Karnataka Deputy Chief Minister D K Shivakumar...,Home,Karnataka Deputy Chief Minister D K Shivakumar...
105371,781129,Forbes,"Tom Sanderson, Senior Contributor, \n Tom Sand...",FC Barcelona Guarantees $77.6 Million Champion...,FC Barcelona have guaranteed at least $77.6 mi...,https://www.forbes.com/sites/tomsanderson/2023...,https://imageio.forbes.com/specials-images/ima...,2023-11-29 08:41:18,FC Barcelona have guaranteed at least $767.6 m...,Home,FC Barcelona have guaranteed at least $767.6 m...
105372,781235,NPR,Brigid McCarthy,Three hospitals ignored her gravely ill fiancé...,"Forty years ago, Sarah Lubarsky came home from...",https://www.npr.org/2023/11/29/1215016001/heal...,https://media.npr.org/assets/img/2023/11/23/sa...,2023-11-29 10:01:12,The photo from David and Sarah Lubarsky's wedd...,Home,The photo from David and Sarah Lubarsky's wedd...
105373,781240,Forbes,"Gary Stern, Contributor, \n Gary Stern, Contri...",Kerber’s Farm: Bringing Farm To Table To Manha...,"A farmstand in Long Island, Kerber’s Farms has...",https://www.forbes.com/sites/garystern/2023/11...,https://imageio.forbes.com/specials-images/ima...,2023-11-29 13:44:33,Kerbers Farm: Bringing Farm To Table To Manhat...,Home,Kerber’s Farm: Bringing Farm To Table To Manha...


In [ ]:
#print (dat["full_content"][3])

In [ ]:
#print (dat["content"][3])

In [ ]:
#keywords =input("Enter the keywords to search").split(",")
keywords=["Himalayan","glacier"]

In [ ]:
masks = [dat["full_content"].str.contains(kw, case=False) for kw in keywords]

In [ ]:
masks

[0          True
 1           NaN
 2           NaN
 3          True
 4           NaN
           ...  
 105370    False
 105371    False
 105372    False
 105373    False
 105374    False
 Name: full_content, Length: 105375, dtype: object,
 0          True
 1           NaN
 2           NaN
 3         False
 4           NaN
           ...  
 105370    False
 105371    False
 105372    False
 105373    False
 105374    False
 Name: full_content, Length: 105375, dtype: object]

In [ ]:
mask = masks[0]
for m in masks[1:]:
    mask = mask | m

In [ ]:
result = dat[mask]
result

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,full_content
0,89541,NaN,International Business Times,Paavan MATHEMA,UN Chief Urges World To 'Stop The Madness' Of ...,UN Secretary-General Antonio Guterres urged th...,https://www.ibtimes.com/un-chief-urges-world-s...,https://d.ibtimes.com/en/full/4496078/nepals-g...,2023-10-30 10:12:35.000000,UN Secretary-General Antonio Guterres urged th...,Nepal,UN Secretary-General Antonio Guterres urged th...
3,89545,NaN,The Indian Express,Editorial,Sikkim warning: Hydroelectricity push must be ...,Ecologists caution against the adverse effects...,https://indianexpress.com/article/opinion/edit...,https://images.indianexpress.com/2023/10/edit-...,2023-10-06 01:20:24.000000,At least 14 persons lost their lives and more ...,Nepal,At least 14 persons lost their lives and more ...
15,89570,NaN,CNA,NaN,China expands climate change surveillance on H...,BEIJING: China has set up weather stations on ...,https://www.channelnewsasia.com/sustainability...,https://onecms-res.cloudinary.com/image/upload...,2023-10-06 08:37:56.000000,BEIJING: China has set up weather stations on ...,Nepal,BEIJING: China has set up weather stations on ...
18,89577,NaN,CNA,NaN,CNA wins gold in Best Use of Video category at...,SINGAPORE: For its first-hand look at the dire...,https://www.channelnewsasia.com/singapore/cna-...,https://onecms-res.cloudinary.com/image/upload...,2023-10-19 09:37:00.000000,SINGAPORE: For its first-hand look at the dire...,Nepal,SINGAPORE: For its first-hand look at the dire...
22,89586,NaN,International Business Times,AFP News,"At Least 10 Dead, 102 Missing In India Glacial...",Indian rescue teams searched on Thursday for 1...,https://www.ibtimes.com/least-10-dead-102-miss...,https://d.ibtimes.com/en/full/4490737/handout-...,2023-10-05 09:00:39.000000,Indian rescue teams searched on Thursday for 1...,Nepal,Indian rescue teams searched on Thursday for 1...
...,...,...,...,...,...,...,...,...,...,...,...,...
103957,754490,al-jazeera-english,Al Jazeera English,Al Jazeera,How rat miners rescued workers from Indian tun...,"After machines broke down, rat miners successf...",https://www.aljazeera.com/news/2023/11/29/how-...,https://www.aljazeera.com/wp-content/uploads/2...,2023-11-29 10:02:31,Forty-one construction workers. Seventeen days...,News,"After machines broke down, rat miners successf..."
104306,759697,al-jazeera-english,Al Jazeera English,Al Jazeera,Kashmir students accused of terror for ‘celebr...,Police have arrested seven students and charge...,https://www.aljazeera.com/news/2023/11/29/kash...,https://www.aljazeera.com/wp-content/uploads/2...,2023-11-29 08:26:44,Authorities in Indian-administered Kashmir hav...,Technology,Police have arrested seven students and charge...
104627,764960,NaN,International Business Times,Pitcha DANGPRASITH and Mohd Imran KHAN in Bihar,"After 17 Days Trapped In Tunnel, India Workers...",After 17 days trapped inside a collapsed Himal...,https://www.ibtimes.com/after-17-days-trapped-...,https://d.ibtimes.com/en/full/4501386/workers-...,2023-11-29 06:45:31,After 17 days trapped inside a collapsed Himal...,Food,After 17 days trapped inside a collapsed Himal...
104688,766206,the-times-of-india,The Times of India,PTI,"After Uttarakhand tunnel rescue, Congress seek...",Congress general secretary Jairam Ramesh said ...,https://economictimes.indiatimes.com/news/indi...,"https://img.etimg.com/thumb/msid-105589642,wid...",2023-11-29 09:04:29,A day after 41 workers were rescued from the S...,Climate,TNN A day after 41 workers were rescued from t...


In [ ]:
counts = result["full_content"].value_counts()

In [ ]:
counts

The giant oaks are turning copper; the hawthorns are burnished-marigold and most of the chestnut trees are vibrant with amber hues. Autumn might have arrived late this year in Windsor Great Park, but it’s turned up in majestic style. As the leaves crunch underfoot and float down from crooked branches above, their golden, sunset colours add a fleeting spectacle to this ancient landscape. Nature is all around the hotel. Tapping into Mother Nature’s playground, Fairmont Windsor Park is found in prime position – alongside the royal park itself. At this time of year, crisp autumnal walks take you through the hotel’s 40 acres of gardens, and into Windsor Great Park’s 4,800 acres of forests, grasslands and meadows. Immaculate grounds are part of the attraction. To arrive at the hotel by car, you cross a scenic bridge and past a lake that is elegant with spurting fountains. On the front lawn, brass sculptures of galloping horses give a nod to the history of the land and they also act as a prec

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
summarizer = pipeline('summarization',model="facebook/bart-large-cnn")

In [ ]:
Article=counts.idxmax()
len(Article)

6074

In [ ]:
L=Article.split()
L=L[0:200]
len(L)

200

In [ ]:
Articlen=" ".join(L)

In [ ]:

len(Articlen)

1178

In [ ]:
print(summarizer(Articlen))

[{'summary_text': "The Fairmont Windsor Park is located in prime position alongside the royal park itself. The hotel's 40 acres of gardens and 4,800 acres of forests, grasslands and meadows are part of the attraction. To arrive at the hotel by car, you cross a scenic bridge and past a lake that is elegant with spurting fountains."}]
